In [19]:
import pandas as pd
import re

def parse_bet_text(text):
    # Clean and prepare the text
    lines = [line.strip() for line in text.split('\n') if line.strip()]

    data = []
    i = 0
    while i < len(lines):
        line = lines[i]

        if line.startswith("Run Line"):
            bet_type = "Spread"
            i += 1
            while i < len(lines) and lines[i] not in ["Total", "Moneyline"]:
                parts = lines[i].split()
                team_outcome = parts[0]
                value = re.search(r'[+-]\d+(\.\d+)?', lines[i]).group() if re.search(r'[+-]\d+(\.\d+)?', lines[i]) else None
                odds = lines[i + 1] if re.match(r'^[+-]\d+$', lines[i + 1]) else None
                data.append([bet_type, team_outcome, value, odds])
                i += 2
            continue

        elif line.startswith("Total"):
            bet_type = "Total"
            i += 1
            while i < len(lines) and (lines[i].startswith("OVER") or lines[i].startswith("UNDER")):
                parts = lines[i].split()
                team_outcome = parts[0]
                value = parts[1] if re.match(r'\d+(\.\d+)?', parts[1]) else None
                i += 1
                odds = lines[i] if re.match(r'^[+-]\d+$', lines[i]) else None
                data.append([bet_type, team_outcome, value, odds])
                i += 1
            continue

        elif line.startswith("Moneyline"):
            bet_type = "Money Line"
            i += 1
            while i < len(lines) and lines[i] not in ["Run Line", "Total"]:
                team_outcome = lines[i]
                odds = lines[i + 1] if re.match(r'^[+-]\d+$', lines[i + 1]) else None
                data.append([bet_type, team_outcome, None, odds])
                i += 2
            continue

        i += 1
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=['Bet Type', 'Team/Outcome', 'Value', 'Odds'])
    return df

# Example usage
text = """
Run Line
ATHLETICS -1.5
-110
ANGELS +1.5
-120
Total
OVER 14.5
-140
UNDER 14.5
+110
Moneyline
ATHLETICS
-225
ANGELS
+175
"""

df = parse_bet_text(text)
print(df)


     Bet Type Team/Outcome Value  Odds
0      Spread    ATHLETICS  -1.5  -110
1      Spread       ANGELS  +1.5  -120
2       Total         OVER  14.5  -140
3       Total        UNDER  14.5  +110
4  Money Line    ATHLETICS  None  -225
5  Money Line       ANGELS  None  +175
